<a href="https://colab.research.google.com/github/jvishnuvardhan/Code_snippets/blob/master/OD_Tutorial_V3_retinanet_spinenet_coco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Object Detection

In [ ]:
# Import main libraries
!pip uninstall -y opencv-python
!pip install -U -q "tensorflow>=2.9.0" "tf-models-official"
import tensorflow as tf
import tensorflow_datasets as tfds
!pip install opencv-python-headless==4.5.2.52
import tensorflow_models as tfm
from official.vision.ops.preprocess_ops import normalize_image, resize_and_crop_image
# import tensorflow_hub as hub

Found existing installation: opencv-python 4.6.0.66
Uninstalling opencv-python-4.6.0.66:
  Successfully uninstalled opencv-python-4.6.0.66
     |████████████████████████████████| 578.0 MB 16 kB/s 
     |████████████████████████████████| 2.2 MB 47.2 MB/s 
     |████████████████████████████████| 438 kB 36.0 MB/s 
     |████████████████████████████████| 1.7 MB 56.5 MB/s 
     |████████████████████████████████| 5.9 MB 55.0 MB/s 
     |████████████████████████████████| 1.3 MB 40.2 MB/s 
     |████████████████████████████████| 238 kB 57.4 MB/s 
     |████████████████████████████████| 352 kB 50.2 MB/s 
     |████████████████████████████████| 43 kB 1.2 MB/s 
     |████████████████████████████████| 116 kB 35.5 MB/s 
     |████████████████████████████████| 99 kB 9.3 MB/s 
     |████████████████████████████████| 38.2 MB 179 kB/s 
     |████████████████████████████████| 5.9 MB 39.6 MB/s 
     |████████████████████████████████| 1.1 MB 47.0 MB/s 
     |████████████████████████████████| 636 kB 52.5 M

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [ ]:
# Import helper libraries
import pprint
import tempfile
from IPython import display
import matplotlib
import matplotlib.pyplot as plt
import os
import io
import pathlib
import cv2
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen
from pprint import pprint # for printing 

## Visualization tools

To visualize the images with the proper detected boxes, we will use the TensorFlow Object Detection API. To install it we will clone the repository.

In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3483, done.
remote: Counting objects: 100% (3483/3483), done.
remote: Compressing objects: 100% (2912/2912), done.
remote: Total 3483 (delta 907), reused 1491 (delta 518), pack-reused 0
Receiving objects: 100% (3483/3483), 46.91 MiB | 20.62 MiB/s, done.
Resolving deltas: 100% (907/907), done.


In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Reading package lists...
Building dependency tree...
Reading state information...
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1696118 sha256=b2d6b54ee227de70d17173a0dca4aaa242a7c8f048be37c64ec7160bd0076ae1
  Stored in directory: /tmp/pip-ephem-wheel-cache-ooo6ty1m/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=4bd9cd42b7fabe89971f89edc195408b60293c50bbe60d22034f60bfa416efc5
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580



  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

## Utilities

Run the following cell to create some utils that will be needed later:

- Helper method to load an image
- Map of Model Name to TF Hub handle

In [ ]:
# Inputs to preprocess functions
MEAN_RGB = (0.485 * 255, 0.456 * 255, 0.406 * 255)
STDDEV_RGB = (0.229 * 255, 0.224 * 255, 0.225 * 255)
CENTER_CROP_FRACTION = 0.875

def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)


def inverse_normalize_image(image,
                    offset=(0.485, 0.456, 0.406),
                    scale=(0.229, 0.224, 0.225)):
  """Normalizes the image to zero mean and unit variance."""
  with tf.name_scope('inverse_normalize_image'):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    scale = tf.constant(scale)
    scale = tf.expand_dims(scale, axis=0)
    scale = tf.expand_dims(scale, axis=0)
    image *= scale

    offset = tf.constant(offset)
    offset = tf.expand_dims(offset, axis=0)
    offset = tf.expand_dims(offset, axis=0)
    image += offset
    return image


def build_inputs_for_segmentation(image, input_image_size, MEAN_RGB, STDDEV_RGB):
  """Builds segmentation model inputs for serving."""
  # Normalizes image with mean and std pixel values.
  image = normalize_image(image, offset=MEAN_RGB, scale=STDDEV_RGB)
  image, _ = resize_and_crop_image(
      image,
      input_image_size,
      padded_size=input_image_size,
      aug_scale_min=1.0,
      aug_scale_max=1.0)
  return image



In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# """Semantic segmentation configuration definition."""
import dataclasses
import os
from typing import List, Optional, Union

import numpy as np
from official.core import exp_factory

In [ ]:
# from absl.testing import parameterized
# from official import vision
from official.core import config_definitions as cfg


In [ ]:
import official
print(dir(official))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'common', 'core', 'modeling', 'nlp', 'projects', 'vision']


In [ ]:
# Initialize experimental config
# exp_config = tfm.core.exp_factory.get_exp_config('seg_deeplabv3_pascal')
# exp_config = tfm.core.exp_factory.get_exp_config('retinanet_resnetfpn_coco')
exp_config = tfm.core.exp_factory.get_exp_config('retinanet_spinenet_coco')
# exp_config = tfm.core.exp_factory.get_exp_config('retinanet_resnetfpn_coco')

# tfds_name = 'cifar10'
print(exp_config.task.model.num_classes) # 91 classes

91


In [ ]:
print(exp_config.as_dict().keys())

dict_keys(['task', 'trainer', 'runtime'])


In [ ]:
# defining category labels
# category_index={1: {'id': 1, 'name': 'Car'},
#  2: {'id': 2, 'name': 'Bus'}}
category_index={1: {'id': 1, 'name': 'Platelets'},
 2: {'id': 2, 'name': 'RBC'},3: {'id': 3, 'name': 'WBC'}}

In [ ]:
# Configure model
exp_config.task.model.num_classes = 2
# exp_config.task.model.backbone.resnet.model_id = 18

# Configure training and testing data
batch_size = 1

# exp_config.task.train_data.input_path = '/content/gdrive/MyDrive/OD_SmallSet_coco/output/train.tfrecord'
# exp_config.task.train_data.input_path = '/content/gdrive/MyDrive/OD_MediumSet_coco/output/train.tfrecord'
exp_config.task.train_data.input_path = '/content/gdrive/MyDrive/BCCD/train/cells.tfrecord'

exp_config.task.train_data.dtype = 'float32'
exp_config.task.train_data.global_batch_size = batch_size

# exp_config.task.validation_data.input_path = '/content/gdrive/MyDrive/OD_SmallSet_coco/output/train.tfrecord'
# exp_config.task.validation_data.input_path = '/content/gdrive/MyDrive/OD_MediumSet_coco/output/train.tfrecord'
exp_config.task.validation_data.input_path = '/content/gdrive/MyDrive/BCCD/train/cells.tfrecord'
exp_config.task.validation_data.dtype = 'float32'
exp_config.task.validation_data.global_batch_size = batch_size

# exp_config.task.annotation_file ='/content/gdrive/MyDrive/OD_SmallSet_coco/annotations.json'
# exp_config.task.annotation_file ='/content/gdrive/MyDrive/OD_MediumSet_coco/train_coco_json.json'
exp_config.task.annotation_file ='/content/gdrive/MyDrive/BCCD/train/_annotations.coco.json'

In [ ]:
logical_device_names = [logical_device.name for logical_device in tf.config.list_logical_devices()]

if 'GPU' in ''.join(logical_device_names):
  print('This may be broken in Colab.')
  device = 'GPU'
elif 'TPU' in ''.join(logical_device_names):
  print('This may be broken in Colab.')
  device = 'TPU'
else:
  print('Running on CPU is slow, so only train for a few steps.')
  device = 'CPU'

if device=='CPU':
  train_steps = 2 # 20
  exp_config.trainer.steps_per_loop = 2 #5
else:
  train_steps=6
  exp_config.trainer.steps_per_loop = 2

exp_config.trainer.summary_interval = 100
exp_config.trainer.checkpoint_interval = train_steps
exp_config.trainer.validation_interval = 100
exp_config.trainer.validation_steps =  2 #ds_info.splits['test'].num_examples // batch_size
exp_config.trainer.train_steps = train_steps
exp_config.trainer.optimizer_config.learning_rate.type = 'cosine'
exp_config.trainer.optimizer_config.learning_rate.cosine.decay_steps = train_steps
exp_config.trainer.optimizer_config.learning_rate.cosine.initial_learning_rate = 0.1
exp_config.trainer.optimizer_config.warmup.linear.warmup_steps = 100
exp_config.task.model.detection_generator.max_classes_per_detection = exp_config.task.model.num_classes
exp_config.runtime.mixed_precision_dtype = None

Running on CPU is slow, so only train for a few steps.


In [ ]:
# Check the default parameters in the experimental config
pprint(exp_config.as_dict()) 


{'runtime': {'all_reduce_alg': None,
             'batchnorm_spatial_persistent': False,
             'dataset_num_private_threads': None,
             'default_shard_dim': -1,
             'distribution_strategy': 'mirrored',
             'enable_xla': False,
             'gpu_thread_mode': None,
             'loss_scale': None,
             'mixed_precision_dtype': None,
             'num_cores_per_replica': 1,
             'num_gpus': 0,
             'num_packs': 1,
             'per_gpu_thread_count': 0,
             'run_eagerly': False,
             'task_index': -1,
             'tpu': None,
             'tpu_enable_xla_dynamic_padder': None,
             'worker_hosts': None},
 'task': {'annotation_file': '/content/gdrive/MyDrive/BCCD/train/_annotations.coco.json',
          'differential_privacy_config': None,
          'export_config': {'cast_detection_classes_to_float': False,
                            'cast_num_detections_to_float': False,
                            'out

In [ ]:
# Setting up the Strategy
logical_device_names = [logical_device.name for logical_device in tf.config.list_logical_devices()]

if exp_config.runtime.mixed_precision_dtype == tf.float16:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')

if 'GPU' in ''.join(logical_device_names):
  distribution_strategy = tf.distribute.MirroredStrategy()
elif 'TPU' in ''.join(logical_device_names):
  tf.tpu.experimental.initialize_tpu_system()
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='/device:TPU_SYSTEM:0')
  distribution_strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
  print('Warning: this will be really slow.')
  distribution_strategy = tf.distribute.OneDeviceStrategy(logical_device_names[0])

print("Done")


Done


In [ ]:
def show_batch(images): # labels, predictions=None
  # plt.figure(figsize=(10, 10))
  min = images.numpy().min()
  max = images.numpy().max()
  delta = max - min

  for i in range(1):
    plt.subplot(3, 3, i + 1)
    plt.imshow((images[i]-min) / delta)
    # if predictions is None:
    #   plt.title(label_info.int2str(labels[i]))
    # else:
    #   if labels[i] == predictions[i]:
    #     color = 'g'
    #   else:
    #     color = 'r'
    #   plt.title(label_info.int2str(predictions[i]), color=color)
    plt.axis("off")

In [ ]:
# model = task.build_model()

In [ ]:
# out = model(feats)

In [ ]:
with distribution_strategy.scope():
  model_dir = tempfile.mkdtemp()
  task = tfm.core.task_factory.get_task(exp_config.task, logging_dir=model_dir)
  print(task)

# tf.keras.utils.plot_model(task.build_model(), show_shapes=True)

In [ ]:
# import orbit
# strategy = tf.distribute.get_strategy()

In [ ]:
# dataset = orbit.utils.make_distributed_dataset(strategy, task.build_inputs,
#                                                    exp_config.task.train_data)
# iterator = iter(dataset)
# feats, labels = next(iterator)

In [ ]:
# print(feats.shape)
# print(labels.keys())

In [ ]:
# print(labels['box_targets'])

In [ ]:
# plt.figure(figsize=(10, 10))
# for images, labels in task.build_inputs(exp_config.task.train_data).take(1):
#   print(f'images.shape: {str(images.shape):16}  images.dtype: {images.dtype!r}')
#   # print(f'labels.shape: {str(labels.shape):16}  labels.dtype: {labels.dtype!r}')
#   # print()
#   # # print(labels)
#   # print(labels.keys())
#   # plt.figure(figsize=(10, 10))
#   # # plt.imshow(images[0])
#   show_batch(images)

In [ ]:
# labels['image_info']

In [ ]:
model, eval_logs = tfm.core.train_lib.run_experiment(
    distribution_strategy=distribution_strategy,
    task=task,
    mode='train_and_eval',
    params=exp_config,
    model_dir=model_dir,
    run_post_eval=True)

loading annotations into memory...
Done (t=1.00s)
creating index...
index created!
restoring or initializing model...
initialized model.
train | step:      0 | training until step 2...


InvalidArgumentError: ignored

In [ ]:
# Plotting metrics
for key, value in eval_logs.items():
  if tf.is_tensor(value):
    print(f'{key:20}: {value.numpy():.3f}')

In [ ]:
from official.vision.serving import export_saved_model_lib
import official.core.train_lib

In [ ]:
# Saving and exporting the trained model
export_saved_model_lib.export_inference_graph(
    input_type='image_tensor',
    batch_size=1,
    input_image_size=[640, 640],
    params=exp_config,
    checkpoint_path=tf.train.latest_checkpoint(model_dir),
    export_dir='./export/')

## Visualization of an original image

In [ ]:
image_path= '/content/gdrive/MyDrive/OD_MediumSet_coco/images/10647071755_c864af9a57_c.jpg' # car
# image_path= '/content/gdrive/MyDrive/OD_MediumSet_coco/images/1302119263_9da3dc23eb_c.jpg'
image_np = load_image_into_numpy_array(image_path)
plt.figure(figsize=(24,32))
plt.imshow(image_np[0]/255.)
plt.show()

In [ ]:
# defining category labels
category_index={1: {'id': 1, 'name': 'Car'},
 2: {'id': 2, 'name': 'Bus'}}

In [ ]:
input_size = (640, 640)

In [ ]:
# apply pre-processing functions which were applied during training the model
image_for_prediction = build_inputs_for_segmentation(np.float32(image_np[0]), input_size, MEAN_RGB, STDDEV_RGB)
image_for_prediction = tf.expand_dims(image_for_prediction, axis=0)
image_for_prediction.get_shape()

In [ ]:
# display pre-processed image
plt.figure(figsize=(24,32))
plt.imshow(image_for_prediction[0])
plt.show()

## Doing the inference

To do the inference we just need to call our model on the given input image.

In [ ]:
# Importing SavedModel
imported = tf.saved_model.load('./export/')
model_fn = imported.signatures['serving_default']

In [ ]:
image_for_prediction_int8 = tf.cast(image_for_prediction, dtype = tf.uint8)
# running inference
results = model_fn(image_for_prediction_int8)

# different object detection models have additional results
# all of them are explained in the documentation
result = {key:value.numpy() for key,value in results.items()}
print(result.keys())

In [ ]:
results['detection_scores']

In [ ]:
np.max(image_for_prediction_int8)

In [ ]:
model_fn.output_dtypes

In [ ]:
results['num_detections']

## Visualizing the results

Here is where we will need the model to show the detection boxes from the inference step.

In [ ]:
height = model_fn.structured_input_signature[1]['inputs'].shape[1]
width = model_fn.structured_input_signature[1]['inputs'].shape[2]
input_size = (height, width)

In [ ]:
print(input_size)

In [ ]:
height, width = 640, 640

In [ ]:
# selecting parameters for visualization
label_id_offset = 0
min_score_thresh =0.9
use_normalized_coordinates=True
if use_normalized_coordinates:
  # Normalizing detection boxes
  result['detection_boxes'][0][:,[0,2]] /= height
  result['detection_boxes'][0][:,[1,3]] /= width

In [ ]:
image_np_resize = cv2.resize(image_np[0], input_size[::-1], interpolation = cv2.INTER_AREA)

In [ ]:
# Visualizing detection boxes with scores
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_for_prediction_int8,#.astype(np.uint8), #new_image[0] image_np_resize
      result['detection_boxes'][0],
      (result['detection_classes'][0] + label_id_offset).astype(int),
      result['detection_scores'][0],
      category_index=category_index,
      use_normalized_coordinates=use_normalized_coordinates,
      max_boxes_to_draw=200,
      min_score_thresh=.9,
      agnostic_mode=False,
      instance_masks=None, #result['detection_masks_reframed'],#.get('detection_masks_reframed', None),
      line_thickness=2)

plt.figure(figsize=(24,32))
plt.imshow(image_for_prediction_int8)
plt.show()

In [ ]:
result['detection_classes']

In [ ]:
result['detection_boxes'][0]

In [ ]:
result['detection_scores'][0]